# **panel** Notebook

Just click on the panel icon (blue square).

In [ ]:
import pandas as pd

In [ ]:
data_url = "https://cdn.jsdelivr.net/gh/holoviz/panel@master/examples/assets/occupancy.csv"
data = pd.read_csv(data_url, parse_dates=["date"]).set_index("date")
data.head()

# Data Visualization
For simplicity we will be using Matplotlib. But you can use any of your favorite plotting libraries.

In [ ]:
import numpy as np
from matplotlib.figure import Figure

In [ ]:
primary_color = "#0072B5"
secondary_color = "#94EA84"

In [ ]:
def mpl_plot(avg, highlight):
    fig = Figure(figsize=(10,5))
    ax = fig.add_subplot()
    avg.plot(ax=ax, c=primary_color)
    if len(highlight):
        highlight.plot(style="o", ax=ax, c=secondary_color)
    return fig

In [ ]:
def find_outliers(variable="Temperature", window=20, sigma=10, view_fn=mpl_plot):
    avg = data[variable].rolling(window=window).mean()
    residual = data[variable] - avg
    std = residual.rolling(window=window).std()
    outliers = np.abs(residual) > std * sigma
    return view_fn(avg, avg[outliers])

find_outliers()

## Data Exploration
Panel can also help you with **interactive data exploration**.

In [ ]:
import panel as pn
pn.extension(sizing_mode="stretch_width")

In [ ]:
# Widgets
variable = pn.widgets.RadioBoxGroup(
    name="Variable", value="Temperature", options=list(data.columns), margin=(-10, 5, 10, 10)
)
window = pn.widgets.IntSlider(name="Window", value=20, start=1, end=60)

In [ ]:
# Reactive Functions
reactive_outliers = pn.bind(find_outliers, variable, window, 10)

In [ ]:
# Layouts
settings = pn.Column(pn.pane.Markdown("Variable", margin=10), variable, window)
pn.Row(pn.WidgetBox(settings, sizing_mode="fixed", width=200), pn.panel(reactive_outliers, sizing_mode="scale_width"), margin=10)

## Layout the App

In [ ]:
template = pn.template.FastListTemplate(
    site="Panel",
    title="Getting Started Example",
    sidebar=[pn.pane.Markdown("Variable"), variable, window],
    main=[pn.panel(reactive_outliers, sizing_mode="scale_width")],
    accent_base_color=primary_color,
    header_background=primary_color,
)
template.servable();

**End of notebook**